# **Max Clique problem using genetic algorithm.**
as per Research paper <br>
25 August 2024 <br>
copyright: ©Pollob Ray(pollob.cray@gmail.com)

**Import Statements**

In [ ]:
import random
import numpy as np
import re # to extract value from a line

**Tuning Parameters**

In [ ]:
#For Genetic Problem
chromosome_length = None #value will be number of node
population_size = 50
generations = 1000  #number of iteration
threshold = 0.5
crossover_rate = 0.8
mutation_rate = 0.1

**Read File**

In [ ]:
my_file = open('/content/sample_data/1.clq', 'r') #load file

clique_line = my_file.readline()      # read first line
node_edge_line = my_file.readline()   # read second line

clique_size = int( re.findall(r'\b\d+\b', clique_line)[0]); # extract first number as max clique size
node = int( re.findall(r'\b\d+\b', node_edge_line)[0]);     # extract first number as number of node

**Set up Global Results**

In [ ]:
global_max_found = 0
global_results_max = list()
global_results_less = list()

**Set up Global min fitness Holders**

In [ ]:
global_min_fitness_1 = 99999
global_min_fitness_2 = 99999

global_min_fitness_1_index = -1
global_min_fitness_2_index = -1

**Initialization & creation adjacency matrix for graph**

In [ ]:
#now Define value of Chromosome size
chromosome_length = node #chromosome length is equal to number of nodes in the graph

adj_matrix = [[0] * node for _ in range(node)] #create 2D matrix and initialize it with zeros

while True:  #initilize adjacency matrix
        next_line = my_file.readline().strip()

        if not next_line:
            break  # Exit loop if no more lines are left

        u = int(re.findall(r'\b\d+\b', next_line)[0]) -1  #1 less numbered
        v = int(re.findall(r'\b\d+\b', next_line)[1]) -1  #1 less numbered

        adj_matrix[u][v] = 1  # 1 indicates there is edge
        adj_matrix[v][u] = 1  # 1 indicates there is edge

# **Genetic Algorithm Steps**

**Chomosome Generation**

In [ ]:
def chromosome_generation(length):
  #chromosome = [random.choice([0, 1]) for _ in range(length)] #when no threshold
  chromosome = list()

  for i in range(length):
        random_number = random.random()  # Generate a random float between 0 and 1
        if random_number >= threshold:
            chromosome.append(1)
        else:
            chromosome.append(0)

  return chromosome

**1. Population Generation**

In [ ]:
def population_generation(chromosome_length,population_size):
  population = [chromosome_generation(chromosome_length) for _ in range(population_size)]
  return population

**a node is connected to a graph or not check**

In [ ]:
def is_connected(u, nodes):
    return all(adj_matrix[u][v] for v in nodes) #check there 'u' is connected to the all nodes or not

**Clique Finding from Graph**

In [ ]:
def subset_clique_find(individual):
  #clicue problem
  #find clique size

  ver = [i for i, x in enumerate(individual) if x == 1] #get vertices that perticipated in that chromosome
  vertices = sorted(ver)

  max_found = 0 #max clique size is found
  results = list() # results contains only clique with max_found size

  for i in range(len(vertices)):
    u = vertices[i]
    result = list()
    result.append(u)

    for j in range(i+1,len(vertices)): # start from i+1 to length-1
      #Break the loop
      if len(result) + (len(vertices) - j) < max_found: #if there is no chnace to make clique of larger size than already found
        result.clear()
        break

      v = vertices[j]

      if not is_connected(v,result):
        continue
      else:
        result.append(v)

    #Max clique found value update
    if  len(result)>0:
      if len(result) > max_found:
        max_found = len(result)
        results = [result]
      #elif len(result) == max_found:
      #  if result not in results:
      #   results.extend([result])
  return results, max_found

In [ ]:
def subset_clique_find_1(individual):
  #clicue problem
  #find clique size

  ver = [i for i, x in enumerate(individual) if x == 1] #get vertices that perticipated in that chromosome
  vertices = sorted(ver)

  max_found = 0 #max clique size is found
  results = list() # results contains only clique with max_found size
  result = list()
  for i in range(len(vertices)):
    u = vertices[i]
    #result = list()
    #result.append(u)

    #for j in range(i+1,len(vertices)): # start from i+1 to length-1
      #Break the loop
     # if len(result) + (len(vertices) - j) < max_found: #if there is no chnace to make clique of larger size than already found
      #  result.clear()
       # break

     # v = vertices[j]

    if i==0:
      result.append(u)
    else:
      if not is_connected(u,result):
        result.clear()
      else:
        result.append(u)

    #Max clique found value update
  if  len(result)>0:
    if len(result) > max_found:
      max_found = len(result)
      results = [result]
      #elif len(result) == max_found:
      #  if result not in results:
      #   results.extend([result])
  return results, max_found

**Fitness Value calculation**

In [ ]:
def fitness_calculation(individual):

  global global_max_found
  global global_results_less
  global global_results_max

  results, max_found = subset_clique_find_1(individual)

  if max_found > global_max_found:
    global_max_found = max_found
    global_results_less = global_results_max
    global_results_max = results


  return max_found

**2. Parents Selection**

In [ ]:
# Rank Based Selection
def rank_selection(probabilities):
    r = random.random()
    cumulative_sum = 0
    for i, p in enumerate(probabilities):
        cumulative_sum += p
        if r <= cumulative_sum:
            return i

def parents_selection_rank_based(population):
  #define rank based selection
  n = population_size
  matrix = [[fitness_calculation(population[i]), i] for i in range(n)] #first field is fitness value second one is index

  # Sorting the matrix based on the first field
  matrix.sort(key=lambda x: x[0])

  ######################## Survivor Selection (Fitness Based Selection) #####################################
  global global_min_fitness_1
  global global_min_fitness_2
  global global_min_fitness_1_index
  global global_min_fitness_2_index

  global_min_fitness_1_index = matrix[0][1] #when sort in ascending
  global_min_fitness_2_index = matrix[1][1]

  global_min_fitness_1 = matrix[0][0]
  global_min_fitness_2 = matrix[1][0]
  ###########################################################################################################

  ranks = list(range(1, n + 1))  # Rank 1 for the worst, n for the best

  # Calculate the total sum of ranks
  total_ranks = sum(ranks)

  # Calculate the probability for each rank
  probabilities = [rank / total_ranks for rank in ranks]

  # Select two parents based on rank
  parent1_index = rank_selection(probabilities)
  parent2_index = rank_selection(probabilities)

  # Output the indices of the selected parents
  parent1_actual_index = matrix[parent1_index][1]
  parent2_actual_index = matrix[parent2_index][1]

  parents = [population[parent1_actual_index], population[parent2_actual_index]]

  return parents #, parents_index

def parents_selection(population):
  return parents_selection_rank_based(population)

**3. Crossover (multipoint)**

In [ ]:
def crossover(parent1, parent2):
  if random.random() <= crossover_rate:
    points = random.sample(range(1, len(parent1)), 2)
    point1 = 0
    point2 = 0
    if points[0] > points[1]:
      point1 = points[1]
      point2 = points[0]
    else:
      point1 = points[0]
      point2 = points[1]

    child1 = parent1[:point1]+parent2[point1:point2]+parent1[point2:]
    child2 = parent2[:point1]+parent1[point1:point2]+parent2[point2:]

    return child1,child2

  else:
    return parent1,parent2

**4. Mutation (Swap)**

In [ ]:
# Swap based Mutation
def mutation(individual1, individual2):
  # mutated_index1 = random.randint(0, len(individual1) - 1)
  # individual1[mutated_index1] = 1 - individual1[mutated_index1] #flip the point
  # mutated_index2 = random.randint(0, len(individual2) - 1)
  # individual2[mutated_index2] = 1 - individual2[mutated_index2] #flip the point

  if random.random() <= mutation_rate:
    points = random.sample(range(1, len(individual1)), 2)
    point1 = 0
    point2 = 0
    if points[0] > points[1]:
      point1 = points[1]
      point2 = points[0]
    else:
      point1 = points[0]
      point2 = points[1]

    fitness1 = fitness_calculation(individual1)
    fitness2 = fitness_calculation(individual2)

    temp_individual1 = individual1
    temp = temp_individual1[point1]
    temp_individual1[point1] = temp_individual1[point2]
    temp_individual1[point2] = temp

    if fitness_calculation(temp_individual1) > fitness1:
      individual1 = temp_individual1

    points = random.sample(range(1, len(individual1)), 2)
    point1 = 0
    point2 = 0
    if points[0] > points[1]:
      point1 = points[1]
      point2 = points[0]
    else:
      point1 = points[0]
      point2 = points[1]

    temp_individual2 = individual2
    temp = temp_individual2[point1]
    temp_individual2[point1] = temp_individual2[point2]
    temp_individual2[point2] = temp

    if fitness_calculation(temp_individual2) > fitness2:
      individual2 = temp_individual2

  return individual1, individual2

**5. Survivor Selection (Fitness Based)**

In [ ]:
def survivor_selection():
  return global_min_fitness_1_index, global_min_fitness_2_index

**Calculating Distance**

In [ ]:
def calculate_distance(individual1,individual2):
  #humming distance
  distance = sum([a ^ b for a, b in zip(individual1, individual2)])

  return distance

**6. Replacement**

In [ ]:
def replacement(population, cp1, cp2):
  index_1, index_2 = survivor_selection()
  population[index_1] = cp1 #p1
  population[index_2] = cp2 #p2

  return population

**Return Global Result**

In [ ]:
def get_results():
  results = global_results_max #+ global_results_less
  return results

**Genetic Method Controller**

In [ ]:
def genetic_algorithm():
  #initialize population
  population = population_generation(chromosome_length,population_size)

  for generation in range(generations):
    #select parents
    parents = parents_selection(population)
    parent_1 = parents[0]
    parent_2 = parents[1]

    #perform crossover
    child1, child2 = crossover(parent_1, parent_2)

    #perform mutation
    child_p1, child_p2 = mutation(child1, child2)

    #perfom deterministic crowding niching technique
    population = replacement(population, child_p1, child_p2)

**Calling & Printing Result**

In [ ]:
genetic_algorithm()
solutions = get_results()

print("Max Clique Size is : ",clique_size)
print("found: ",global_max_found)
print("Solutions are")
if(solutions): # checking solutions is empty or not
   for sol in solutions:
      print(f"Value : {len(sol)} -> {sol}")



